# Which Cross-Validation is Best? (Discussion)

For this discussion I used the medical cost personal dataset (https://www.kaggle.com/datasets/mirichoi0218/insurance/data).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.express as px
import sklearn
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.utils import shuffle
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from Modules import cprint

C:\Users\matth\AppData\Local\Temp\ipykernel_59600\876166824.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data = pd.read_csv('data/insurance.csv')

In [3]:
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


To test group k-fold cross validation (https://towardsdatascience.com/why-you-should-never-use-cross-validation-4360d42456ac), want to remove one region entirely, run validation procedures on other regions, and then test on final region with all methods.

In [4]:
data.region.value_counts()

region
southeast    364
southwest    325
northwest    325
northeast    324
Name: count, dtype: int64

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


Roughly equal values for each region. Remove northeast from dataset and train on the rest of the dataset.

Will test following aspects of the model:
- Number of polynomial features (up to degree = 3)
- Ridge regression coefficients (alpha ranging from 0 to 10)

# Prep Dataset
Before any training, need to numerically encode categorical variables.

In [6]:
print(data.sex.value_counts())
print(data.smoker.value_counts())

sex
male      676
female    662
Name: count, dtype: int64
smoker
no     1064
yes     274
Name: count, dtype: int64


Sex and smoker have one value each, so can use one-hot encoding with binary enabled

In [7]:
ohe = OneHotEncoder(sparse_output=False, drop='if_binary')
sex_ohe = pd.DataFrame(ohe.fit_transform(data[['sex']]), columns = ['sex_ohe'], index = data.index)
smoker_ohe = pd.DataFrame(ohe.fit_transform(data[['smoker']]), columns = ['smoker_ohe'], index = data.index)
model_data = pd.concat([data, sex_ohe, smoker_ohe], axis=1)

In [8]:
# Should always be two lines for each
cprint(model_data[['sex', 'sex_ohe']].value_counts(), 'blue')
cprint(model_data[['smoker', 'smoker_ohe']].value_counts(), 'green')

sex     sex_ohe
male    1.0        676
female  0.0        662
Name: count, dtype: int64
smoker  smoker_ohe
no      0.0           1064
yes     1.0            274
Name: count, dtype: int64


For sex, 'male' has been coded as 1, and for smoker 'no' has been coded as 0.

Next, we will drop extra columns (sex and smoker are now superfluous) and prep the test dataset (where region = 'northeast')

In [9]:
model_data = model_data.drop(['sex', 'smoker'], axis=1)

In [10]:
test_data = model_data.query("region == 'northeast'")
train_data = model_data.query("region != 'northeast'")
cprint(test_data.region.value_counts(), 'blue')
cprint(train_data.region.value_counts(), 'green')

region
northeast    324
Name: count, dtype: int64
region
southeast    364
southwest    325
northwest    325
Name: count, dtype: int64


Because we will be running k-fold where k=3 and we want to sort by region, reorder test_data to have an equal amount of all.

In [11]:
new_index = list(range(0, 325)) + list(range(364, 364+325+325))
new_index
train_data = train_data.sort_values('region')
train_data = train_data.reset_index(drop=True).loc[new_index,:]
train_data.region.value_counts() # should be 325 for all

region
northwest    325
southeast    325
southwest    325
Name: count, dtype: int64

In [12]:
model_data.columns

Index(['age', 'bmi', 'children', 'region', 'charges', 'sex_ohe', 'smoker_ohe'], dtype='object')

In [13]:
X_train, y_train = train_data.drop('charges', axis=1), train_data['charges']
X_test, y_test = test_data.drop('charges', axis=1), test_data['charges']

In [15]:
cprint(X_train.head(), 'green')
cprint(y_train.head(), 'blue')
print(X_train.region.value_counts())

   age     bmi  children     region  sex_ohe  smoker_ohe
0   61  29.070         0  northwest      0.0         1.0
1   24  26.790         1  northwest      1.0         0.0
2   48  36.670         1  northwest      1.0         0.0
3   19  39.615         1  northwest      0.0         0.0
4   46  19.855         0  northwest      1.0         0.0
0    29141.36030
1    12609.88702
2    28468.91901
3     2730.10785
4     7526.70645
Name: charges, dtype: float64
region
northwest    325
southeast    325
southwest    325
Name: count, dtype: int64


# K-fold cross validation: 10
Below, we test k-fold cross validation. We will test with *k* = 10.

Steps:
- Create pipeline for testing, which will transform our model as necessary.
- Create parameter grid
- Run kfold CV

In [16]:
np.random.seed(42)
kfold_ind_shuffle = shuffle(X_train.index)
print(kfold_ind_shuffle)

kfold_X_train = X_train.drop('region', axis=1).loc[kfold_ind_shuffle,:]
kfold_y_train = y_train[kfold_ind_shuffle]

kfold_X_test = X_test.drop('region', axis=1)
kfold_y_test = y_test

kfold_X_train.head()

Index([199, 828, 174, 506,  66, 897, 672, 637, 809, 158,
       ...
       121, 653,  20, 739,  71, 106, 270, 899, 474, 102],
      dtype='int64', length=975)


,age,bmi,children,sex_ohe,smoker_ohe
199,32,29.735,0,0.0,0.0
828,34,38.000,3,0.0,0.0
174,52,31.730,2,0.0,0.0
506,18,43.010,0,1.0,0.0
66,32,27.835,1,1.0,0.0


In [17]:
kfold_pipe = Pipeline([
    ('poly', PolynomialFeatures(include_bias=False)),
    ('standard', StandardScaler()),
    ('ridge', Ridge())
])

kfold_param_dict = {
    'poly__degree': [1, 2, 3],
    'ridge__alpha': np.linspace(0, 10, 50)
}

np.random.seed(100)
kfold_grid = GridSearchCV(estimator=kfold_pipe, param_grid=kfold_param_dict, scoring='neg_mean_squared_error', cv=10)
kfold_grid.fit(kfold_X_train, kfold_y_train)

C:\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=9.96845e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('poly',
                                        PolynomialFeatures(include_bias=False)),
                                       ('standard', StandardScaler()),
                                       ('ridge', Ridge())]),
             param_grid={'poly__degree': [1, 2, 3],
                         'ridge__alpha': array([ 0.        ,  0.20408163,  0.40816327,  0.6122449 ,  0.81632653,
        1.02040816,  1.2244898 ,  1.42857143,  1.63265306,  1.83673469,
        2.04081633,  2.24489796,  2.44897959,  2.65306122,  2.8571...
        4.08163265,  4.28571429,  4.48979592,  4.69387755,  4.89795918,
        5.10204082,  5.30612245,  5.51020408,  5.71428571,  5.91836735,
        6.12244898,  6.32653061,  6.53061224,  6.73469388,  6.93877551,
        7.14285714,  7.34693878,  7.55102041,  7.75510204,  7.95918367,
        8.16326531,  8.36734694,  8.57142857,  8.7755102 ,  8.97959184,
        9.18367347,  9.3877551 ,  9.59183673,  9.79591837, 10.        ])},
             scoring='neg_mean_squared_error')

In [18]:
kfold_best_model = kfold_grid.best_estimator_
kfold_best_selector = kfold_best_model.named_steps['ridge']
kfold_best_params = kfold_grid.best_params_
kfold_best_score = kfold_grid.best_score_
kfold_best_coefs = kfold_best_selector.coef_

print(kfold_best_params)
kfold_best_model

{'poly__degree': 3, 'ridge__alpha': 0.20408163265306123}


Pipeline(steps=[('poly', PolynomialFeatures(degree=3, include_bias=False)),
                ('standard', StandardScaler()),
                ('ridge', Ridge(alpha=0.20408163265306123))])

Get R^2 values for training and test models

In [19]:
kfold_y_train_pred = kfold_best_model.predict(kfold_X_train)
kfold_train_r2 = r2_score(kfold_y_train, kfold_y_train_pred)
print(f"Train R^2: {kfold_train_r2}")
print(f"MSE: {-kfold_best_score}")

Train R^2: 0.8579196064230067
MSE: 23655395.6665989


In [20]:
kfold_y_test_pred = kfold_best_model.predict(kfold_X_test)
kfold_test_r2 = r2_score(kfold_y_test, kfold_y_test_pred)
print(f"Train R^2: {kfold_test_r2}")

Train R^2: 0.7935671809483413


Overprediction by a bit, but still in ballpark.

# K-fold Cros Validation: 3
Below, we sort by region and then test with cv=3. Because each region consists of 1/3 of the data, we will not shuffle the data this time.

In [21]:
kfold_X_train = X_train.drop('region', axis=1)
kfold_y_train = y_train

kfold_X_test = X_test.drop('region', axis=1)
kfold_y_test = y_test

kfold_X_train.head()

,age,bmi,children,sex_ohe,smoker_ohe
0,61,29.070,0,0.0,1.0
1,24,26.790,1,1.0,0.0
2,48,36.670,1,1.0,0.0
3,19,39.615,1,0.0,0.0
4,46,19.855,0,1.0,0.0


In [22]:
kfold_pipe = Pipeline([
    ('poly', PolynomialFeatures(include_bias=False)),
    ('standard', StandardScaler()),
    ('ridge', Ridge())
])

kfold_param_dict = {
    'poly__degree': [1, 2, 3],
    'ridge__alpha': np.linspace(0, 10, 50)
}

np.random.seed(100)
kfold_grid = GridSearchCV(estimator=kfold_pipe, param_grid=kfold_param_dict, scoring='neg_mean_squared_error', cv=3)
kfold_grid.fit(kfold_X_train, kfold_y_train)

C:\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=1.29247e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('poly',
                                        PolynomialFeatures(include_bias=False)),
                                       ('standard', StandardScaler()),
                                       ('ridge', Ridge())]),
             param_grid={'poly__degree': [1, 2, 3],
                         'ridge__alpha': array([ 0.        ,  0.20408163,  0.40816327,  0.6122449 ,  0.81632653,
        1.02040816,  1.2244898 ,  1.42857143,  1.63265306,  1.83673469,
        2.04081633,  2.24489796,  2.44897959,  2.65306122,  2.85714...
        4.08163265,  4.28571429,  4.48979592,  4.69387755,  4.89795918,
        5.10204082,  5.30612245,  5.51020408,  5.71428571,  5.91836735,
        6.12244898,  6.32653061,  6.53061224,  6.73469388,  6.93877551,
        7.14285714,  7.34693878,  7.55102041,  7.75510204,  7.95918367,
        8.16326531,  8.36734694,  8.57142857,  8.7755102 ,  8.97959184,
        9.18367347,  9.3877551 ,  9.59183673,  9.79591837, 10.        ])},
             scoring='neg_mean_squared_error')

In [23]:
kfold_best_model = kfold_grid.best_estimator_
kfold_best_selector = kfold_best_model.named_steps['ridge']
kfold_best_params = kfold_grid.best_params_
kfold_best_score = kfold_grid.best_score_
kfold_best_coefs = kfold_best_selector.coef_

print(kfold_best_params)
kfold_best_model

{'poly__degree': 2, 'ridge__alpha': 2.0408163265306123}


Pipeline(steps=[('poly', PolynomialFeatures(include_bias=False)),
                ('standard', StandardScaler()),
                ('ridge', Ridge(alpha=2.0408163265306123))])

In [24]:
kfold_y_train_pred = kfold_best_model.predict(kfold_X_train)
kfold_train_r2 = r2_score(kfold_y_train, kfold_y_train_pred)
print(f"Train R^2: {kfold_train_r2}")
print(f"MSE: {-kfold_best_score}")

Train R^2: 0.8513808415619606
MSE: 24424916.053919982


In [25]:
kfold_y_test_pred = kfold_best_model.predict(kfold_X_test)
kfold_test_r2 = r2_score(kfold_y_test, kfold_y_test_pred)
print(f"Train R^2: {kfold_test_r2}")

Train R^2: 0.7961563783740752


# Conclusion
In this instance, both cross-validation techniques decided on the same model.